In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os
import numpy as np

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [ ]:
##  configuration options
config_file = '../testingdir/parameters.xml'

cfg = Parameters(config_file)

radius=0.01
longitude=0.1
time_range=[cfg.start_time, cfg.end_time]
use_grid=True
ylim=None
use_cache=False
verbose=True

specie = 'electrons'

autoselect = True

x_axis_label = r'$\mathit{t (s)}$'
y_axis_label = r'$\mathit{T (eV)}$'

plot_name = r'$\mathbf{Temperature-time \enspace Dependency}\enspace(T)$'

In [ ]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         data_root=cfg.data_root,
                         fullframe_size=[cfg.number_r_grid , cfg.number_z_grid],
                         fpds=cfg.frames_per_file, 
                         use_cache=use_cache,
                         verbose=verbose)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

In [ ]:
# get data
start_frame = None # cfg.get_frame_number_by_timestamp(time_range[0])
end_frame = None # 460 # cfg.get_frame_number_by_timestamp(time_range[1])
row_number = cfg.get_row_by_radius(radius)
col_number = cfg.get_col_by_longitude(longitude)

data = []

for probe in cfg.probes:
    if (probe.type == 'dot') and (probe.specie == specie) and (probe.r_start == row_number) and (probe.z_start == col_number):
        start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
        end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
        if probe.component == 'T': data = reader.get_frame_range_dot('T/{}'.format(probe.specie), row_number, col_number, start_frame, end_frame)

if len(data) == 0 and autoselect:
    for probe in cfg.probes:
        if (probe.type == 'col') and (probe.specie == specie) and (probe.z_start == col_number):
            start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            if probe.component == 'T': data = reader.get_frame_range_col('T/{}'.format(probe.specie), col_number, start_frame, end_frame)[:, row_number]
            break
        elif (probe.type == 'row') and (probe.specie == specie) and (probe.r_start == row_number):
            start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            if probe.component == 'T': data = reader.get_frame_range_row('T/{}'.format(probe.specie), row_number, start_frame, end_frame)[:, col_number]
            break
        elif (probe.type == 'frame') and (probe.specie == specie) and (probe.r_start <= row_number) and (probe.z_start <= col_number) and (probe.r_end >= row_number) and (probe.z_end >= col_number):
            start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            shape = [probe.r_start, probe.z_start, probe.r_end, probe.z_end]
            if probe.component == 'T':
                for piece in range(start_frame, end_frame, cfg.frames_per_file):
                    data[piece - start_frame:piece - start_frame + cfg.frames_per_file - 1] = reader.get_frame_range('T/{}'.format(probe.specie), shape, piece, piece + cfg.frames_per_file)[:, row_number - probe.r_start, col_number - probe.z_start]
data = np.nan_to_num(data)

In [ ]:
data_timeline = np.linspace(time_range[0], time_range[1], len(data))

# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=use_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

# add subplots
plot_obj = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# set y-limits
if ylim is not None:
    plot_obj.set_ylim(ylim)

In [ ]:
# add data
plot_obj.plot(data_timeline, data)

plot.show()